In [1]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200218014242-0000
KERNEL_ID = 3237bcff-442a-49b0-9d67-722577e86d83
--2020-02-18 01:42:45--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet [following]
--2020-02-18 01:42:45--  https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving to: 'hmp.parquet'

100%[======================================>] 932,997     --.-K/s   in 0.06s   

2020-02-18 01:42:45 (14.0 MB/s

In [5]:
df.show()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [30]:
# Feature engineer one column, energy, here it will be called label as below

df_energy = spark.sql("""
        select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label,  class\
        from df
        group by class       

""")

In [31]:
df_energy.createOrReplaceTempView('df_energy')

In [32]:
# create a join to include the energy column

df_join = spark.sql("""
            select * from df\
            inner join df_energy
            on df.class = df_energy.class           
            
""")

df_join.show(5)

+---+---+---+--------------------+-----------+-----------------+-----------+
|  x|  y|  z|              source|      class|            label|      class|
+---+---+---+--------------------+-----------+-----------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
+---+---+---+--------------------+-----------+-----------------+-----------+
only showing top 5 rows



In [24]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

from pyspark.ml.regression import LinearRegression

vectorAssembler = VectorAssembler(inputCols=['x', 'y', 'z'], outputCol='features')
normalizer = Normalizer(inputCol='features', outputCol='features_norm', p=1.0)

In [25]:
# Create the model
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [27]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, normalizer, lr])

In [34]:
model = pipeline.fit(df_join)
predictions = model.transform(df_join)

In [35]:
# acquire the linear regression which is No: 3 in the pipeline stages

model.stages[2].summary.r2

0.03259100556263628